In [7]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import sys
import os
from dotenv import load_dotenv
import datetime

load_dotenv()

True

In [8]:
engine = sa.create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
			.format(host=os.getenv('host'), db=os.getenv('db'), user=os.getenv('uname'), pw=os.getenv('password')))

class Loan:
    ObservationNmb: int
    maturity_dt: datetime.date
    origination_dt: datetime.date
    lender_status: int
    all_pmts: np.ndarray
    
    def __init__(self, nmb, m_dt, o_dt, l_st,pmts):
        self.ObservationNmb = nmb
        self.maturity_dt = m_dt
        self.origination_dt = o_dt
        self.all_pmts = pmts
        self.lender_status = l_st
        
    def find_payoff_date(self):
        pass
 


In [6]:
def get_ids()-> list:
    with engine.connect() as con:
            con.execute('SET GLOBAL innodb_buffer_pool_size=2147483648;')
            # Get ObservationNmb's for all non-revolver loans
            query = """ SELECT elipsmisc7afoia.ObservationNmb, repymnttbl7afoia.MaturityDt, elipsmisc7afoia.LoanFundedDt, loantbl7afoia.LenderStatusCd FROM elipsmisc7afoia
                    JOIN repymnttbl7afoia
                    ON repymnttbl7afoia.ObservationNmb = elipsmisc7afoia.ObservationNmb
                    JOIN loantbl7afoia
                    ON loantbl7afoia.ObservationNmb = elipsmisc7afoia.ObservationNmb
                    WHERE RevolvingLineofCreditCd = 'Y' AND YEAR(LoanFundedDt) = 2000 ;"""
                    
            print("aggregating loan ids...\n")
            rows = con.execute(query)
            return [list(ele) for ele in rows.fetchall()]

my_ids = get_ids()

aggregating loan ids...



In [13]:
def build_cohort(loan_dict, year):
    cohort = []
    for key in loan_dict:
        if loan_dict[key].origination_dt.year == year:
            cohort.append(loan_dict[key])

    return cohort

def do_principal_pmts(cohrt_ls: list):
    """WHENEVER YOU TEST A CHANGE --- CHANGE cohrt_ls to cohrt_ls[0:2] and see how it looks"""
    master_principal = []
    print('parsing all payment strings...')
    for x in (cohrt_ls):
        hist = x.all_pmts
        hist = np.insert(hist, 0, x.maturity_dt, axis=1)
        hist = np.insert(hist, 0, x.origination_dt, axis=1)      
        hist = np.insert(hist, 0, x.ObservationNmb, axis=1)
        # hist_apprv_inc = [ (hist[:, 1] == 151)&(hist[:,2] == '6002') ]
        hist_1510 = hist[ hist[:, 5]== '1510', :]
        hist_6031 = hist[ hist[:, 5]== '6031', :]
        principal_pmts = np.append(hist_1510, hist_6031, axis=0)
        order = principal_pmts[:, 3].argsort()
        principal_pmts = principal_pmts[order]
        master_principal.append(principal_pmts)
    print('returning master principal pmt structure')
    return master_principal


def build_df_list(prepayment_arr, cols):
    df_dict = {}
    for d in prepayment_arr:
        df_x = pd.DataFrame(data=d, columns= cols).drop_duplicates().reset_index().drop(columns='index')
        if df_x.empty == True:
            continue
        else:
            obs=df_x.ObservationNmb.values[0]
            df_dict[obs]= (df_x.drop_duplicates().reset_index().drop(columns='index'))
            
    return df_dict

In [11]:
loans = {}

print("looping...\n")
with engine.connect() as con:    
    for id in (my_ids):
        con.execute('SET GLOBAL innodb_buffer_pool_size=2147483648;')
        query = f"""SELECT EffectiveDt, TransacionCd, GeneralLedgerCd, TransactionAmt, TransactionBalanceAmt FROM master_fin WHERE ObservationNmb = \"{id[0]}\" """
        result = con.execute(query)
        rows = np.array([list(ele) for ele in result])
        loans[id[0]] = Loan(id[0], id[1], id[2], id[3], rows)

looping...



In [12]:
cohort_map = {}

for 

dict_keys(['1469637', '1469646', '1469648', '1469651', '1469656', '1469659', '1480820', '1480830', '1480842', '1480853', '1480856', '1480861', '1480862', '1480867', '1480875', '1480895', '1480897', '1480899', '1480904', '1480930', '1480963', '1480996', '1481057', '1481062', '1481080', '1481099', '1481103', '1481105', '1481111', '1481112', '1481115', '1481128', '1481201', '1481242', '1481243', '1481245', '1481267', '1481268', '1481269', '1481274', '1481315', '1481339', '1481352', '1481426', '1481430', '1481454', '1481468', '1481502', '1481537', '1481556', '1481565', '1481567', '1481569', '1481574', '1481614', '1481628', '1481653', '1481732', '1481737', '1481742', '1481749', '1481752', '1481797', '1481812', '1481876', '1481886', '1481897', '1481900', '1481912', '1481952', '1481955', '1481962', '1481970', '1481983', '1482015', '1482023', '1482045', '1482047', '1482060', '1482078', '1482088', '1482132', '1482149', '1482160', '1482162', '1482166', '1482197', '1482198', '1482201', '1482224',